#### Notebook to download BID/ASK ticks from Interactive Brokers API

In [ ]:
# Read in list of tickers
import pandas as pd
at = pd.read_csv('..\\..\\data\\available_tickers.csv')
at['0']

In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import ListOfHistoricalTickBidAsk

import threading
import time
import datetime as dt
import os

class IBapi(EWrapper, EClient):
    def __init__(self):
        self.last_time = None
        self.current_symbol = None
        EClient.__init__(self, self)
        called_back = False
        
    def historicalTicksBidAsk(self, reqId: int, ticks: ListOfHistoricalTickBidAsk, done: bool):
        self.called_back=True
        if done:

            if len(ticks) > 0:
                print('lenticks',len(ticks))
                data = [
                    (t.time,
                     t.time,
                     t.tickAttribBidAsk if t.tickAttribBidAsk.bidPastLow or t.tickAttribBidAsk.askPastHigh else None,
                     t.priceBid, 
                     t.priceAsk, 
                     t.sizeBid, 
                     t.sizeAsk)
                    for t in ticks
                ]
                ticks_df = pd.DataFrame(
                    data,
                    columns=['Time', 'TimeStamp', 'TickAttribBidAsk', 'PriceBid', 'PriceAsk', 'SizeBid', 'SizeAsk'],
                )
                ticks_df['Time']=pd.to_datetime(ticks_df['Time'], unit='s', utc=True)
                ticks_df['Time'] = ticks_df['Time'].dt.tz_convert('Europe/Oslo')

                end_time = ticks_df['Time'].max()
                print('endtime',end_time)
                
                filename = self.current_symbol + '_' + end_time.strftime('%Y-%m') + '_BIDASK.csv'
                ticks_df.to_csv('..\\..\\data\\TICKS\\' + filename, mode='a', header=not os.path.exists(filename), index=False)
            else:
                end_time = dt.datetime.fromtimestamp(self.last_time)

            if len(ticks) < 1000: # hop to next day
                lt = dt.datetime.combine(end_time, dt.datetime.min.time())
                self.last_time = dt.datetime.timestamp(lt + dt.timedelta(days=1))
            else:
                self.last_time = dt.datetime.timestamp(end_time) + 1
                

app = IBapi()
app.connect('127.0.0.1', 7496, 123)


def run_loop():
    app.run()

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

# LOOP THROUGH ALL TICKERS
for ticker in at['0']:
    print(ticker)
    #Create contract object
    stock = Contract()
    stock.symbol = ticker
    stock.secType = 'STK'
    stock.exchange = 'OSE'
    stock.currency = 'NOK'
    
    start_time = dt.datetime(2022, 3, 1, 9, 0, 0)
    while start_time < dt.datetime(2022, 3, 31, 23, 59, 59):# dt.datetime.now():
        app.called_back = False
        app.current_symbol = stock.symbol

        print('starttime',start_time.strftime('%Y%m%d %H:%M:%S'))
        app.reqHistoricalTicks(2, stock, start_time.strftime('%Y%m%d %H:%M:%S'), "", 1000, "BID_ASK", 0, False, [])
        time.sleep(2.7) #sleep to allow enough time for data to be returned
        if not app.called_back:
            app.last_time = app.last_time + 1
        if not app.last_time:
            start_time = start_time + dt.timedelta(days=1)
        else:
            start_time = dt.datetime.fromtimestamp(app.last_time)

app.disconnect()